# Final Project Foundamentals of Data Science
### Solar Eclipse phases classification
#### Luca Moresca, Nicholas Suozzi, Valerio Santini

---------
-   **Titolo del progetto:** Classificazione delle fasi dell'eclissi solare 
-   **Autori:** Luca Moresca, Nicholas Suozzi, Valerio Santini
-   **Descrizione del problema:** Fornire una breve panoramica del problema della classificazione delle fasi dell'eclissi solare. Evidenziare l'importanza dello studio delle eclissi solari per la comprensione dell'interazione tra Sole e Terra.
-   **Obiettivo del progetto:** Definire chiaramente l'obiettivo del progetto, ovvero sviluppare un modello in grado di classificare accuratamente le diverse fasi di un'eclissi solare da immagini.


### Abstract


In [ ]:
if True:
    %pip install -qqq numpy scipy matplotlib pandas scikit-learn seaborn tqdm

In [ ]:
%matplotlib inline
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import os
from pathlib import Path
import json
from PIL import Image
from scipy.signal import convolve2d
import math
from scipy.ndimage import convolve1d
from skimage.metrics import structural_similarity as ssim
from skimage.transform import resize
from skimage import io, color
from skimage.measure import label, regionprops
from skimage.draw import rectangle
from skimage.filters import threshold_otsu
from typing import List, Tuple, Optional, Dict, Union
%load_ext autoreload
%autoreload 2

# sets up pandas table display
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

## 1. Dataset Analysis

The “Great American Eclipse” in 2017 offered a unique opportunity for US citizens across the entire continent to experience the awe-inspiring changes that occur during rare total solar eclipses (TSEs). The goal of the 2017 Eclipse Megamovie was to obtain enough pictures of the Great American Total Solar Eclipse to create a movie of the solar corona in order to better study it. Over 2,000 volunteers submitted 50,000 images and a movie was made only a few hours after the Moon's shadow left the U.S. A few months later another, better aligned and more closely edited, movie was released. The Eclipse Megamovie team of scientists processed images to create High Dynamic Range (HDR) images to capture the structures within the high dynamic range of the solar corona from the Sun's photosphere out ~3 solar radii.
https://eclipsemegamovie.org

Eclipse megamovie is a project being carried out by a group of researchers in collaboration with NASA. The focus of this project is to make a video that can facilitate research into a rare event such as the eclipse that occurred in the United States in 2017. To realise this project, American citizens were asked to send in photographs taken during the eclipse. The dataset therefore contains semi-professional and amateur images, taken under conditions that are not always optimal.

In [ ]:
# Print the .json file with the label.
with open('eclipse-megamovie/label_num_to_phase_map.json', 'r') as file:
    data = json.load(file)
print(json.dumps(data, indent=4))


In [ ]:
# Print the .csv file with labels and images names.
df = pd.read_csv("eclipse-megamovie/train.csv")
print(df)

The .json file relates the eclipse phases to the corresponding labels in the ‘train.csv’ file, which will help the training phase by already having a correspondence between label and image. The csv file then contains a classification of the various training images useful for the model. 

In [ ]:
# Plot of some images in the train dataset

img1 = mpimg.imread('eclipse-megamovie/train/00a0f91f0e7ece88ebaeab2a2050cdc0d72c5ea0b56c601dfaf800077df44c51.jpg')
img2 = mpimg.imread('eclipse-megamovie/train/00a4af4cc3f2a667c80331051a733e8dae2477d682d7dcef3e895f6f923dd86a.jpg')
img3 = mpimg.imread('eclipse-megamovie/train/00c11b16fd6426a88039c8b27e2511e2e770b65e0eb9daab720d3e6ca996bc06.jpg')
img4 = mpimg.imread('eclipse-megamovie/train/001e6704a60798684f1656713b593e6a56de81474e7173a31936589cd62f13e8.jpg')
img5 = mpimg.imread('eclipse-megamovie/train/4f99a8e1efe0a51fca127bcdabfb1165523dee8730dbba0df5436a7de70dc16e.jpg')

fig, axes = plt.subplots(1, 5, figsize=(16, 4))
fig.tight_layout(pad=3.0)

axes[0].imshow(img1)
axes[0].set_title("Image 1")
axes[0].axis('off')

axes[1].imshow(img2)
axes[1].set_title("Image 2")
axes[1].axis('off')

axes[2].imshow(img3)
axes[2].set_title("Image 3")
axes[2].axis('off')

axes[3].imshow(img4)
axes[3].set_title("Image 4")
axes[3].axis('off')

axes[4].imshow(img5)
axes[4].set_title("Image 5")
axes[4].axis('off')

plt.show()


* As can be seen in the figure above, our dataset consists of various types of images. Some contain eclipses, others do not. The images have different perspectives, are taken with different devices and have different backgrounds. 
This presents us with the challenge of correctly identifying the content of each image accurately, we must be able to classify images that do not contain eclipses as impostors. 
On a numerical level, the dataset consists of a training set and a test set. In addition, there is a .csv file containing the name of some images and a label showing the corresponding eclipse phase. In addition, there is a .json file that explains what each label corresponds to on a phase level. 
There are 496 images in the training set and 140 images in the test set. The dataset is restricted and this will result in an appropriate treatment of the images to improve the performance of the model. 

* Each image has a different resolution, considering that they are obtained with various devices. The eclipse within each image is not always perfectly sharp or centred, making the model more elastic in various cases.


## 2. Preprocessing delle immagini

-   **Spiegazioni:** Descrivere le tecniche di preprocessing delle immagini utilizzate, spiegando la logica alla base di ogni scelta.
-   Ad esempio, se si ridimensionano le immagini, spiegare perché è necessario farlo e quale dimensione si è scelto di utilizzare.
-   Se si convertono le immagini in scala di grigi, giustificare questa scelta.
-   Se si normalizzano i valori dei pixel, spiegare il metodo di normalizzazione utilizzato e perché è importante.
-   **Codice:** Fornire il codice Python utilizzato per il preprocessing delle immagini.
-   **Visualizzazioni:** Mostrare alcuni esempi di immagini pre-elaborate, confrontandole con le immagini originali.


### Suggerimenti per il Preprocessing delle Immagini per la Classificazione delle Fasi di Eclissi Solare

Ecco alcuni suggerimenti per descrivere le tecniche di preprocessing delle immagini che potresti utilizzare per il tuo progetto di classificazione delle fasi di eclissi solare, insieme alla logica alla base di ogni scelta:

*   **Conversione in scala di grigi:** La conversione delle immagini in scala di grigi è una scelta comune nel preprocessing delle immagini, soprattutto quando il colore non è una caratteristica discriminante per il compito in questione. Nel caso della classificazione delle fasi di eclissi solare, le informazioni di colore potrebbero non essere essenziali per distinguere le diverse fasi. La conversione in scala di grigi riduce la complessità dei dati e la quantità di informazioni da elaborare, il che può accelerare i tempi di addestramento del modello e potenzialmente migliorare le sue prestazioni.
*   **Riduzione del rumore:** Le immagini possono contenere rumore che può influenzare negativamente le prestazioni del modello. La riduzione del rumore può essere realizzata utilizzando filtri come il filtro gaussiano.  Applicare un filtro gaussiano prima di altre operazioni di preprocessing può aiutare a migliorare la qualità delle immagini e a rendere più evidenti le caratteristiche importanti per la classificazione.
*   **Downscaling:**  Se le immagini originali hanno una risoluzione molto alta, il downscaling può essere utile per ridurre la quantità di dati da elaborare e accelerare i tempi di addestramento. La dimensione da utilizzare dipenderà dalla risoluzione originale delle immagini e dalla complessità del modello. È importante scegliere una dimensione che conservi le caratteristiche essenziali per la classificazione, evitando al contempo di perdere troppe informazioni.  Assicurati di applicare un filtro gaussiano prima del downscaling per evitare l'aliasing. 
*   **Normalizzazione:** La normalizzazione dei valori dei pixel è un'altra tecnica comune nel preprocessing delle immagini.  Può essere eseguita sottraendo la media e dividendo per la deviazione standard dei valori dei pixel. Questo processo porta tutti i valori dei pixel in un intervallo simile, il che può aiutare a migliorare le prestazioni del modello, soprattutto quando si utilizzano algoritmi di apprendimento automatico sensibili alla scala dei dati.

**Tecniche utili dagli homework:**

*   **Dal Homework 1:** Tutte le tecniche elencate (riduzione del rumore, downscaling, normalizzazione e conversione in scala di grigi) possono essere utili per il tuo progetto.
*   **Dal Homework 2:** La codifica one-hot sarà necessaria per le etichette delle fasi dell'eclissi. La gestione dei valori mancanti potrebbe essere utile se il tuo set di dati contiene immagini incomplete o danneggiate. L'espansione polinomiale e lo scaling potrebbero non essere necessari per il tuo progetto, in quanto sono tecniche più comunemente utilizzate per dati numerici non immagine.

### Tecniche Essenziali di Preprocessing per Immagini di Eclissi Solari Variabili

Dato che si ha a che fare con una vasta gamma di immagini scattate da amatori, che presentano diverse proporzioni, dimensioni del sole e posizioni del sole all'interno dell'immagine, è fondamentale applicare tecniche di preprocessing essenziali per preparare i dati per l'addestramento del modello di classificazione delle fasi dell'eclissi. 

Oltre alle tecniche di base già discusse (conversione in scala di grigi, riduzione del rumore, downscaling e normalizzazione), ecco alcune altre tecniche molto essenziali da considerare:

*   **Ritaglio e ridimensionamento:** Le immagini potrebbero avere bordi o aree irrilevanti che non contribuiscono alla classificazione delle fasi dell'eclissi. Il ritaglio può essere utilizzato per rimuovere queste aree e focalizzare l'attenzione sulla regione di interesse, ovvero il sole e la corona solare. Dopo il ritaglio, tutte le immagini dovrebbero essere ridimensionate a una dimensione uniforme per garantire la compatibilità con il modello. Poiché le proporzioni delle immagini originali possono variare, è possibile utilizzare un ridimensionamento con padding per mantenere le proporzioni originali, aggiungendo bordi neri attorno all'immagine ritagliata per raggiungere la dimensione desiderata. 
*   **Centratura del Sole:** La posizione del sole all'interno dell'immagine può variare notevolmente. Per migliorare le prestazioni del modello, è utile centrare il sole in tutte le immagini. Ciò può essere ottenuto tramite algoritmi di rilevamento degli oggetti o, in casi più semplici, analizzando l'istogramma dell'immagine per identificare la regione più luminosa (presumibilmente il sole) e spostarla al centro dell'immagine.

### Come Rilevare e Centrare il Sole nelle Immagini di Eclissi

Per centrare il sole in un'immagine, un metodo robusto è quello di utilizzare la **combinazione di rilevamento dei bordi, piramide gaussiana e template matching**, tecniche già presenti negli homework forniti.

**Ecco i passaggi**

1. **Rilevamento dei Bordi**

2. **Piramide Gaussiana** 

3. **Template Matching** 

4. **Selezione della Scala Migliore** 

5. **Centratura** 

**Note Aggiuntive**

* Per migliorare ulteriormente la precisione del rilevamento, è possibile utilizzare un template del sole più sofisticato, che tenga conto della corona solare.
* Dopo la centratura, potrebbe essere necessario ritagliare l'immagine per rimuovere eventuali bordi neri aggiunti durante il ridimensionamento con padding.

Questo metodo offre una soluzione efficace per rilevare e centrare il sole in immagini di eclissi solare, utilizzando tecniche già familiari dagli homework e garantendo una buona robustezza e flessibilità.




### Conversione in scala di grigi 

In [ ]:
import cv2

path = "eclipse-megamovie/train/f5b6b5e1fe97a2070a105c2031d1d84d4cc764746dc9d4c5a94db9b099d976a2.jpg"
#path = "eclipse-megamovie/train/00a0f91f0e7ece88ebaeab2a2050cdc0d72c5ea0b56c601dfaf800077df44c51.jpg"
image = cv2.imread(path)
gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# cv2.imwrite('/Users/valeriosantini/Desktop/or.jpg', image)
# cv2.imwrite('/Users/valeriosantini/Desktop/gr.jpg', gray_image)

plt.imshow(gray_image, cmap="gray")
plt.axis('off')
plt.show()




COMMENTARE

### Denoising

In [ ]:
def gauss(sigma: float, filter_size: int) -> Tuple[np.ndarray, np.ndarray]:
    Gx = np.array([])
    k = (filter_size - 1) // 2  
    x = np.arange(-k, k + 1)
    cost = 1/(np.sqrt(2*np.pi)*sigma)
    Gx = np.append(Gx, cost * np.exp(-(x**2)/(2*sigma**2)))
    return Gx, x

def gaussfiltering(img: np.ndarray, sigma: float) -> np.ndarray:
    filter_size = np.round(6 * sigma)

    Gx, _ = gauss(sigma, filter_size)
    out = convolve1d(img, Gx, axis = 1)
    out = convolve1d(out, Gx, axis = 0)
    
    return out

In [ ]:
sigma = 3

smt_img = gaussfiltering(gray_image, sigma)

plt.imshow(smt_img, cmap="gray")
plt.axis('off')
plt.show()

COMMENTARE


### Normalization

In [ ]:
def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

In [ ]:
nrm_img = normalize(smt_img)

The normalize function performs linear normalization of an image to scale pixel values in the range $[0,1]$

### Eclipse Isolation

In [ ]:
def detect_eclipse(image):


    offset = 128
    intensity_threshold = threshold_otsu(image)
    bright_pixels = image > intensity_threshold

    labeled_bright = label(bright_pixels)
    regions = regionprops(labeled_bright)


    if len(regions) == 0:
        print("Nessuna regione luminosa rilevata.")
        return None
    largest_region = max(regions, key=lambda r: r.area)
    minr, minc, maxr, maxc = largest_region.bbox

    height = maxr - minr + offset
    width = maxc - minc + offset
    
    side_length = max(height, width)

    center_r = (minr + maxr) // 2
    center_c = (minc + maxc) // 2

    minr = center_r - side_length // 2
    maxr = center_r + side_length // 2
    minc = center_c - side_length // 2
    maxc = center_c + side_length // 2
    
    if minr < 0:
        maxr += abs(minr)  
        minr = 0
    if maxr > image.shape[0]:
        minr -= maxr - image.shape[0]
        maxr = image.shape[0]

    if minc < 0:
        maxc += abs(minc)   
        minc = 0
    if maxc > image.shape[1]:
        minc -= maxc - image.shape[1]
        maxc = image.shape[1]

    minr = max(0, minr)
    minc = max(0, minc)
    maxr = min(image.shape[0], maxr)
    maxc = min(image.shape[1], maxc)

    cropped_image = image[minr:maxr, minc:maxc]



    return cropped_image


This function identifies and isolates the brightest region of an image by performing a square crop centered on it. It uses Otsu's threshold method to determine the brightest pixels and then identifies the largest connected region using regionprops. After calculating an extended bounding box with an offset, it determines a square crop based on the longest side of the detected area, centering the crop on the bright region.

If the crop exceeds the boundaries of the image, the edges are corrected to keep the crop within the image itself. In the absence of bright regions, the function returns None.

In [ ]:
cropped_img = detect_eclipse(nrm_img)


#Print Histogram
hist, bins = np.histogram(nrm_img.flatten(), bins=256, range=(0, 1))
plt.figure(figsize=(10, 4))
plt.title("Istogramma delle Intensità")
plt.bar(bins[:-1], hist, width=0.005, color='gray')
plt.xlabel("Intensità")
plt.ylabel("Numero di pixel")
plt.show()

#Print Cropped Img
plt.figure(figsize=(12, 6))
plt.title("Immagine Ritagliata")
plt.imshow(cropped_img, cmap='gray')
plt.show()


### Downscaling

In [ ]:
import numpy as np
from skimage.transform import resize

def downscale(
    img: np.ndarray,
    target_resolution: tuple = (512, 512),
    sigma: float = 1
) -> np.ndarray:

    if img.ndim == 3:   
        img_blurred = np.zeros_like(img)
        for c in range(3):
            img_blurred[:, :, c] = gaussfiltering(img[:, :, c], sigma)
    else:   
        img_blurred = gaussfiltering(img, sigma)

    downscaled_img = resize(img_blurred, target_resolution, anti_aliasing=True)

    if img.ndim == 3:  
        downscaled_img = np.clip(downscaled_img * 255, 0, 255).astype(np.uint8)
        img_blurred = np.clip(img_blurred * 255, 0, 255).astype(np.uint8)

    return downscaled_img


The downscale function downscales an image to a specified target resolution, applying Gaussian pre-filtering to reduce artifacts due to sampling.

Main details:

* Accepts an input image (img) and an optional target resolution parameter (target_resolution, default (512, 512)).
* Applies Gaussian filtering (gaussfiltering) to smooth the image before resizing.
* Uses skimage.transform.resize to resize the image, with anti_aliasing option to preserve visual quality.
* Converts filtered and resized images to 8-bit (uint8) format to ensure compatibility and optimize memory.

In [ ]:
downscaled = downscale(cropped_img, target_resolution=(512, 512), sigma=3)

fig, axes = plt.subplots(1, 2, figsize=(12, 8))
plt.gray()
fig.tight_layout()

axes[0].imshow(cropped_img)
axes[0].set_title(f"Original Image {smt_img.shape}")
axes[0].axis('off')
axes[1].imshow(downscaled)
axes[1].set_title(f"Downscaled Image {downscaled.shape}")
axes[1].axis('off')

plt.show()

In [ ]:
def preprocess_image(path: str, target_resolution: tuple = (512, 512), sigma: float = 3, offset: int = 128):
    image = cv2.imread(path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    smt_img = gaussfiltering(gray_image, sigma)
    nrm_img = normalize(smt_img)
    cropped_img = detect_eclipse(nrm_img)
    downscaled_img = downscale(cropped_img, target_resolution, sigma)
    return downscaled_img

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def get_image_paths(folder_path):
    supported_formats = ('.jpg', '.png')
    return [
        os.path.join(folder_path, filename)
        for filename in os.listdir(folder_path)
        if filename.lower().endswith(supported_formats)
    ]

def preprocess_images_in_parallel(folder_path, target_resolution=(512, 512), sigma=3, num_workers=4):
    image_paths = get_image_paths(folder_path)
    
    def process(path):
        return preprocess_image(path, target_resolution, sigma)

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        processed_images = list(executor.map(process, image_paths))
    
    return np.array(processed_images)

def save_to_npy(dataset, output_file):
    np.save(output_file, dataset)

input_folder = "eclipse-megamovie/train"
output_file = "/Users/valeriosantini/Desktop/preprocessed_dataset.npy"
target_resolution = (512, 512)
sigma = 3 

processed_images = preprocess_images_in_parallel(input_folder, target_resolution, sigma, num_workers=8)
save_to_npy(processed_images, output_file)

print(f"Preprocessing completato e dataset salvato in {output_file}.")

In [ ]:
# Carica il file .npy
data = np.load("/Users/valeriosantini/Desktop/preprocessed_dataset.npy")

# Stampa alcune informazioni
print("Shape:", data.shape)
print("Data type:", data.dtype)

# Mostra un esempio
print("Primo elemento (array):")
print(data[0])  # Mostra il primo elemento


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


# Dataset personalizzato
class SolarEclipseDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.data = pd.read_csv(csv_file)  # Carica il file CSV
        self.image_folder = image_folder
        self.target_resolution = target_resolution
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.data.iloc[idx, 0])
        image = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)  # Carica in grayscale
        image = cv2.resize(image, self.target_resolution)  # Ridimensiona l'immagine
        label = self.data.iloc[idx, 1]


        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

# Trasformazioni (normalizzazione e resize)
transform = transforms.Compose([
    transforms.ToTensor(),  # Converte l'immagine in tensore
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalizzazione
])

# Carica il dataset
csv_file = "eclipse-megamovie/train.csv"
image_folder = "eclipse-megamovie/train"
dataset = SolarEclipseDataset(csv_file, image_folder, transform=transform)

# DataLoader per batch
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SolarEclipseCNN(nn.Module):
    def __init__(self, num_classes=8):
        super(SolarEclipseCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # Input: 1x512x512 -> Output: 32x512x512
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Output: 64x512x512
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Riduce dimensione: Output: 64x256x256
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)  # Output: 128x256x256
        self.fc1 = nn.Linear(128 * 64 * 64, 512)  # Flatten -> Fully connected
        self.fc2 = nn.Linear(512, num_classes)  # Output layer

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))  # Output: 128x64x64
        x = x.view(-1, 128 * 64 * 64)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
import torch.optim as optim

# Istanziamento del modello, loss e ottimizzatore
#device = torch.device("mps" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")
model = SolarEclipseCNN(num_classes=8).to(device)
criterion = nn.CrossEntropyLoss()  # Loss per classificazione multiclass
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loop di training
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass e ottimizzazione
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

print("Training completato.")


In [ ]:
# Save the trained model 
model_path = "eclipse-megamovie/solar_net.pth"
torch.save(model.state_dict(), model_path)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Modalità valutazione
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Report di classificazione
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Matrice di confusione
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Now we can try preprocessing with an image given its path

In [ ]:
from PIL import Image, UnidentifiedImageError

class CustomTestDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.image_names = [
            f for f in os.listdir(image_folder) 
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.image_folder, img_name)
        try:
            image = Image.open(img_path).convert("L")  # Converti in scala di grigi
        except UnidentifiedImageError:
            raise ValueError(f"Il file {img_path} non è un'immagine valida.")

        if self.transform:
            image = self.transform(image)

        return image, img_name

    
# Definisci le trasformazioni
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # Ridimensiona le immagini
    transforms.ToTensor(),  # Converti in tensore
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalizza
])

# Carica il dataset
image_folder = "eclipse-megamovie/test"  # Percorso alla tua cartella di test
test_dataset = CustomTestDataset(image_folder=image_folder, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=False)

# Metti il modello in modalità di valutazione
model.eval()

# Predizioni
predictions = []

with torch.no_grad():
    for data in test_loader:
        images, img_names = data
        images = images.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Predici le classi

        # Salva i risultati
        for img_name, label in zip(img_names, predicted.cpu().numpy()):
            predictions.append((img_name, label))

# Stampa l'accuratezza (se hai etichette vere, puoi calcolarla)
print("Predizioni completate.")

# Salva i risultati in un CSV
df = pd.DataFrame(predictions, columns=["Image_Name", "Predicted_Label"])
df.to_csv("eclipse_predictions.csv", index=False)
print("Predizioni salvate in: eclipse_predictions.csv")


In [ ]:
# img_folder = "eclipse-megamovie/train"
# prcs_img = []

# for filename in os.listdir(img_folder):
#     if filename.endswith(".jpg") or filename.endswith(".png"):
#         image_path = os.path.join(img_folder, filename)
#         processed_image = preprocess_image(image_path)
#         #print(filename)
#         image_255 = (processed_image * 255).astype(np.uint8)
#         cv2.imwrite(f'/Users/valeriosantini/Desktop/provapre/{filename}', image_255)

preprocessed_img = preprocess_image("eclipse-megamovie/train/0017d87ef1aeb1cc9003d013e652885755657aef55c22c248da595a3d480b119.jpg")

plt.imshow(preprocessed_img)
plt.axis('off')

## 3. Estrazione delle feature

-   **Istogrammi:Spiegazioni:** Spiegare come gli istogrammi possono essere utilizzati per estrarre feature dalle immagini di eclissi solare. [1]
-   Descrivere come la distribuzione dei pixel nell'istogramma può variare a seconda della fase dell'eclissi.
-   **Codice:** Fornire il codice Python utilizzato per calcolare gli istogrammi.
-   **Visualizzazioni:** Mostrare alcuni esempi di istogrammi per le diverse fasi dell'eclissi.
-   **Altre tecniche (opzionale):** Se si utilizzano altre tecniche di estrazione delle feature, includere una sezione simile a quella degli istogrammi, spiegando e visualizzando i risultati.

### Suggerimento su come procedere

Ecco i passaggi da eseguire, considerando che hai già le funzioni di pre-elaborazione e hai ottenuto immagini quadrate e ridimensionate del sole centrato:

*   **Organizzare il dataset:** Crea una struttura per il tuo dataset, ad esempio un dizionario o un dataframe Pandas, dove ogni elemento contiene:
    *   Percorso dell'immagine
    *   Etichetta della fase dell'eclissi (es. "parziale", "totale", "anulare")
*   **Applicare le funzioni di pre-elaborazione:** Per ogni immagine nel dataset:
    *   Carica l'immagine.
    *   Applica le funzioni di pre-elaborazione esistenti (conversione in scala di grigi, filtro gaussiano, normalizzazione, rilevamento e ritaglio dell'eclissi, ridimensionamento).
*   **Calcolare l'istogramma:** Per ogni immagine pre-elaborata:
    *   Calcola l'istogramma dell'immagine utilizzando una libreria come Matplotlib o OpenCV.
    *   Aggiungi l'istogramma al dataset, associandolo all'immagine corrispondente.
*   **Addestrare il modello:** Utilizza il set di training, composto dagli istogrammi e dalle relative etichette, per addestrare il modello di classificazione.
*   **Valutare il modello:** Utilizza il set di test per valutare le prestazioni del modello addestrato.

Ricorda che puoi anche esplorare l'utilizzo di altre tecniche di estrazione delle feature, come **Edge Detection**, per ottenere feature aggiuntive e migliorare potenzialmente l'accuratezza del modello.  Le fonti che hai fornito offrono informazioni dettagliate su queste tecniche e sulla loro implementazione.


## 4. Scelta e implementazione del modello

-   **Modello scelto:** Indicare chiaramente il modello di classificazione scelto per il progetto (ad esempio, Regressione Logistica, Regressione Softmax, CNN). [4-6]
-   Giustificare la scelta del modello in base alle caratteristiche del problema e del dataset.
-   Fornire una breve descrizione teorica del modello scelto, evidenziandone i principi di funzionamento.
-   **Codice:** Fornire il codice Python utilizzato per implementare il modello.
-   Utilizzare le competenze acquisite durante la realizzazione di "HomeWork.pdf" per implementare il modello in modo efficiente. [7]
-   Commentare il codice in modo chiaro per renderlo comprensibile.
-   **Training del modello:Spiegazioni:** Descrivere la procedura di training del modello, specificando la funzione di loss, l'algoritmo di ottimizzazione e gli iperparametri utilizzati. [8-20]
-   Spiegare come si è divisa il dataset in set di training e di test.
-   **Codice:** Fornire il codice Python utilizzato per il training del modello.
-   **Visualizzazioni:** Mostrare l'andamento della funzione di loss durante il training. [21-27]
-   Plottare anche l'accuratezza sul set di training e sul set di test durante il training.


## 5. Valutazione del modello

-   **Spiegazioni:** Descrivere le metriche utilizzate per valutare le prestazioni del modello. [13, 28-32]
-   Oltre all'accuratezza, considerare altre metriche come precisione, recall e F1-score, soprattutto se si ha uno sbilanciamento di classe.
-   **Codice:** Fornire il codice Python utilizzato per calcolare le metriche di valutazione.
-   **Risultati:** Presentare i risultati ottenuti dal modello sul set di test.
-   Utilizzare tabelle e grafici per visualizzare i risultati in modo chiaro ed efficace.
-   **Discussione:** Analizzare i risultati ottenuti, commentando le prestazioni del modello e le eventuali limitazioni.

## 6. Video delle transizioni dell'eclissi (opzionale) 

-   **Spiegazioni:** Descrivere il processo di creazione del video, includendo il software utilizzato e le scelte creative fatte.
-   Spiegare come si sono selezionate le immagini per il video e come si è creata la sequenza di transizione.
-   **Video:** Includere il video nel notebook Jupyter.

## 7. Conclusioni

-   **Riepilogo:** Riassumere brevemente il lavoro svolto nel progetto.
-   **Risultati principali:** Evidenziare i risultati più importanti ottenuti.
-   **Sviluppi futuri:** Suggerire eventuali miglioramenti o sviluppi futuri del progetto.